# Analysing the basement data to predict house prices

### Pre-process data

In [71]:
#Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [72]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics

from sklearn.metrics import mean_squared_error
from math import sqrt

from catboost import CatBoostRegressor, Pool

In [73]:
data = r'/Users/OliverPan/Desktop/house/train.csv'

In [74]:
df = pd.read_csv(data)

In [75]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [76]:
df.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'Pav

In [77]:
basement_df = df[['Id', "SalePrice", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF"]]

In [78]:
basement_df.head()

,Id,SalePrice,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
0,1,208500,Gd,TA,No,GLQ,706,Unf,0,150,856
1,2,181500,Gd,TA,Gd,ALQ,978,Unf,0,284,1262
2,3,223500,Gd,TA,Mn,GLQ,486,Unf,0,434,920
3,4,140000,TA,Gd,No,ALQ,216,Unf,0,540,756
4,5,250000,Gd,TA,Av,GLQ,655,Unf,0,490,1145


In [79]:
#Filter df
high_corr = basement_df.corr().abs().unstack()
sort = high_corr.sort_values(kind="quicksort")
sort = sort.to_frame()

#Look at correlation that is higher than 0.4 and not 1
sort = sort[sort[0] >= 0.4]
sort = sort[sort[0] != 1]

In [80]:
### Correlation between basement variables
sort

,,0
TotalBsmtSF,BsmtUnfSF,0.415360
BsmtUnfSF,TotalBsmtSF,0.415360
BsmtFinSF1,BsmtUnfSF,0.495251
BsmtUnfSF,BsmtFinSF1,0.495251
BsmtFinSF1,TotalBsmtSF,0.522396
TotalBsmtSF,BsmtFinSF1,0.522396
SalePrice,TotalBsmtSF,0.613581
TotalBsmtSF,SalePrice,0.613581


In [81]:
basement_df.nunique()

Id              1460
SalePrice        663
BsmtQual           4
BsmtCond           4
BsmtExposure       4
BsmtFinType1       6
BsmtFinSF1       637
BsmtFinType2       6
BsmtFinSF2       144
BsmtUnfSF        780
TotalBsmtSF      721
dtype: int64

##### It seems that Total BSMT is the sum of SF1 and SF2 and UnfSF, so we should just keep totals as an indicator of SalePrice

In [82]:
basement_df = basement_df.drop(["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF"], axis = 1)

In [83]:
basement_df.head()

,Id,SalePrice,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF
0,1,208500,Gd,TA,No,GLQ,Unf,856
1,2,181500,Gd,TA,Gd,ALQ,Unf,1262
2,3,223500,Gd,TA,Mn,GLQ,Unf,920
3,4,140000,TA,Gd,No,ALQ,Unf,756
4,5,250000,Gd,TA,Av,GLQ,Unf,1145


In [84]:
basement_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            1460 non-null   int64 
 1   SalePrice     1460 non-null   int64 
 2   BsmtQual      1423 non-null   object
 3   BsmtCond      1423 non-null   object
 4   BsmtExposure  1422 non-null   object
 5   BsmtFinType1  1423 non-null   object
 6   BsmtFinType2  1422 non-null   object
 7   TotalBsmtSF   1460 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 91.4+ KB


##### The rest of the categories have 4-6 value_counts, so we can turn them into categorical columns

In [85]:
basement_df["BsmtQual"] = basement_df["BsmtQual"].astype("str")
basement_df["BsmtCond"] = basement_df["BsmtCond"].astype("str")
basement_df["BsmtExposure"] = basement_df["BsmtExposure"].astype("str")
basement_df["BsmtFinType1"] = basement_df["BsmtFinType1"].astype("str")
basement_df["BsmtFinType2"] = basement_df["BsmtFinType2"].astype("str")

#Previously, we found that BsmtFinType2 has no importance for model
basement_df = basement_df.drop(["Id", "BsmtFinType2"], axis = 1)

### CatBoost

In [86]:
X = basement_df.drop(["SalePrice"], axis = 1)
y = basement_df[["SalePrice"]]

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [88]:
category = ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1"]

In [89]:
cat_boost = CatBoostRegressor(depth=3, learning_rate=0.1, loss_function='RMSE')
cat_boost.fit(X_train, y_train, cat_features=category)

0:	learn: 74680.7491172	total: 5.78ms	remaining: 5.77s
1:	learn: 71400.7687511	total: 7.34ms	remaining: 3.66s
2:	learn: 68440.0873182	total: 9.49ms	remaining: 3.15s
3:	learn: 65742.7035336	total: 16.1ms	remaining: 4.01s
4:	learn: 63544.8970354	total: 19ms	remaining: 3.78s
5:	learn: 61583.9827339	total: 20.6ms	remaining: 3.41s
6:	learn: 60080.8520571	total: 22.7ms	remaining: 3.23s
7:	learn: 58492.3771541	total: 23.8ms	remaining: 2.95s
8:	learn: 57283.3010769	total: 24.8ms	remaining: 2.73s
9:	learn: 56416.5564116	total: 35.5ms	remaining: 3.51s
10:	learn: 55280.1900124	total: 37ms	remaining: 3.33s
11:	learn: 54255.2237761	total: 39.2ms	remaining: 3.23s
12:	learn: 53345.0776301	total: 40.3ms	remaining: 3.06s
13:	learn: 52607.0930016	total: 45.2ms	remaining: 3.18s
14:	learn: 51979.3934843	total: 47.2ms	remaining: 3.1s
15:	learn: 51484.8212304	total: 50.6ms	remaining: 3.11s
16:	learn: 50942.9591074	total: 53ms	remaining: 3.06s
17:	learn: 50467.3452822	total: 54.2ms	remaining: 2.96s
18:	learn

158:	learn: 43488.7441766	total: 366ms	remaining: 1.93s
159:	learn: 43483.7398982	total: 367ms	remaining: 1.93s
160:	learn: 43471.5192241	total: 368ms	remaining: 1.92s
161:	learn: 43467.5349101	total: 373ms	remaining: 1.93s
162:	learn: 43463.9310438	total: 375ms	remaining: 1.92s
163:	learn: 43458.3282484	total: 376ms	remaining: 1.92s
164:	learn: 43453.1625802	total: 377ms	remaining: 1.91s
165:	learn: 43442.4777694	total: 378ms	remaining: 1.9s
166:	learn: 43440.4406095	total: 379ms	remaining: 1.89s
167:	learn: 43430.1368033	total: 380ms	remaining: 1.88s
168:	learn: 43426.3954932	total: 381ms	remaining: 1.87s
169:	learn: 43417.0226937	total: 382ms	remaining: 1.87s
170:	learn: 43392.4799936	total: 383ms	remaining: 1.86s
171:	learn: 43380.7006027	total: 386ms	remaining: 1.86s
172:	learn: 43355.7586591	total: 388ms	remaining: 1.85s
173:	learn: 43344.6273598	total: 389ms	remaining: 1.84s
174:	learn: 43340.1005947	total: 389ms	remaining: 1.84s
175:	learn: 43327.1930605	total: 390ms	remaining:

317:	learn: 41517.7527357	total: 592ms	remaining: 1.27s
318:	learn: 41508.8078152	total: 598ms	remaining: 1.28s
319:	learn: 41504.5010744	total: 600ms	remaining: 1.27s
320:	learn: 41494.3779875	total: 602ms	remaining: 1.27s
321:	learn: 41477.5421965	total: 604ms	remaining: 1.27s
322:	learn: 41455.7348785	total: 605ms	remaining: 1.27s
323:	learn: 41451.8169318	total: 606ms	remaining: 1.26s
324:	learn: 41445.5966422	total: 607ms	remaining: 1.26s
325:	learn: 41432.4498489	total: 612ms	remaining: 1.26s
326:	learn: 41431.9771281	total: 613ms	remaining: 1.26s
327:	learn: 41430.9096306	total: 614ms	remaining: 1.26s
328:	learn: 41427.3118992	total: 615ms	remaining: 1.25s
329:	learn: 41425.2152673	total: 616ms	remaining: 1.25s
330:	learn: 41412.8396390	total: 617ms	remaining: 1.25s
331:	learn: 41411.7918492	total: 618ms	remaining: 1.24s
332:	learn: 41410.2214997	total: 619ms	remaining: 1.24s
333:	learn: 41396.9348128	total: 620ms	remaining: 1.24s
334:	learn: 41392.9184707	total: 621ms	remaining

485:	learn: 39756.0989448	total: 896ms	remaining: 948ms
486:	learn: 39753.3637342	total: 901ms	remaining: 949ms
487:	learn: 39751.3594907	total: 903ms	remaining: 947ms
488:	learn: 39744.4535735	total: 907ms	remaining: 948ms
489:	learn: 39738.7340495	total: 908ms	remaining: 945ms
490:	learn: 39727.9197458	total: 909ms	remaining: 943ms
491:	learn: 39714.7943377	total: 911ms	remaining: 941ms
492:	learn: 39661.3850100	total: 916ms	remaining: 942ms
493:	learn: 39649.4242128	total: 918ms	remaining: 940ms
494:	learn: 39639.4692928	total: 919ms	remaining: 938ms
495:	learn: 39637.3753354	total: 923ms	remaining: 938ms
496:	learn: 39634.4654709	total: 926ms	remaining: 937ms
497:	learn: 39623.4293002	total: 927ms	remaining: 934ms
498:	learn: 39604.4310340	total: 931ms	remaining: 935ms
499:	learn: 39598.6194598	total: 934ms	remaining: 934ms
500:	learn: 39589.9407894	total: 935ms	remaining: 932ms
501:	learn: 39582.3494247	total: 937ms	remaining: 930ms
502:	learn: 39581.7025624	total: 939ms	remaining

654:	learn: 38397.0591527	total: 1.25s	remaining: 658ms
655:	learn: 38389.3934463	total: 1.25s	remaining: 656ms
656:	learn: 38386.0053356	total: 1.25s	remaining: 654ms
657:	learn: 38382.0176558	total: 1.25s	remaining: 652ms
658:	learn: 38372.4500934	total: 1.25s	remaining: 649ms
659:	learn: 38365.8201599	total: 1.26s	remaining: 649ms
660:	learn: 38351.6417104	total: 1.26s	remaining: 646ms
661:	learn: 38341.8410587	total: 1.26s	remaining: 644ms
662:	learn: 38335.5978612	total: 1.26s	remaining: 642ms
663:	learn: 38329.2121348	total: 1.26s	remaining: 639ms
664:	learn: 38314.3413577	total: 1.26s	remaining: 637ms
665:	learn: 38313.5878450	total: 1.26s	remaining: 635ms
666:	learn: 38310.0095719	total: 1.27s	remaining: 632ms
667:	learn: 38296.4071255	total: 1.27s	remaining: 630ms
668:	learn: 38291.9847287	total: 1.27s	remaining: 628ms
669:	learn: 38287.1035767	total: 1.27s	remaining: 626ms
670:	learn: 38282.7467406	total: 1.27s	remaining: 624ms
671:	learn: 38279.1741703	total: 1.27s	remaining

868:	learn: 36990.5477219	total: 1.6s	remaining: 242ms
869:	learn: 36988.2034600	total: 1.61s	remaining: 240ms
870:	learn: 36976.6965024	total: 1.61s	remaining: 238ms
871:	learn: 36972.9900233	total: 1.61s	remaining: 236ms
872:	learn: 36970.7683697	total: 1.61s	remaining: 234ms
873:	learn: 36957.0761653	total: 1.61s	remaining: 232ms
874:	learn: 36953.6187818	total: 1.61s	remaining: 230ms
875:	learn: 36953.2166005	total: 1.61s	remaining: 228ms
876:	learn: 36950.7723207	total: 1.61s	remaining: 226ms
877:	learn: 36950.4924152	total: 1.61s	remaining: 224ms
878:	learn: 36948.4860804	total: 1.61s	remaining: 222ms
879:	learn: 36947.5191531	total: 1.61s	remaining: 220ms
880:	learn: 36942.7072720	total: 1.62s	remaining: 218ms
881:	learn: 36939.1388633	total: 1.62s	remaining: 216ms
882:	learn: 36932.1778746	total: 1.62s	remaining: 214ms
883:	learn: 36928.9136952	total: 1.62s	remaining: 213ms
884:	learn: 36927.1345885	total: 1.62s	remaining: 211ms
885:	learn: 36926.3073238	total: 1.62s	remaining:

In [90]:
y_pred = cat_boost.predict(X_test)

In [91]:
#Results using regressor
df = pd.DataFrame({'Actual': y_test["SalePrice"].to_list(), 'Predicted': list(y_pred.flatten())})
df.head()

,Actual,Predicted
0,200624,208868.507431
1,133000,141662.575921
2,110000,151002.727286
3,192000,180406.282597
4,88000,103649.729129


In [92]:
## We should also look at RMSE
rmse = sqrt(mean_squared_error(y_test, y_pred))
rmse

52626.2496379812

In [93]:
## Normalised RMSE
sqrt(mean_squared_error(y_test, y_pred)) / (y_test["SalePrice"].max() - y_test["SalePrice"].min())

0.07761983722416106

In [94]:
list(cat_boost.get_feature_importance(Pool(X_test, label = y_test, cat_features=category)))

[38.90071168327252,
 1.5007923822496567,
 8.373343396151819,
 7.687886066966812,
 43.537266471359175]

In [95]:
basement_df.drop(["SalePrice"], axis = 1).columns.values

array(['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'TotalBsmtSF'], dtype=object)